# Transfer Learning

https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

1. Load MobileNet (smaller ImageNet pre-trained CNN), without classifier
2. Train classifier with food pictures
3. Fine-tune the last layers and classifier

In [1]:
import matplotlib.pyplot as plt

# Image loading and augmentation
from keras.preprocessing.image import ImageDataGenerator

# Source network
from keras.applications import mobilenet

# Target network's classifier
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout

# training
from keras.callbacks import TensorBoard, EarlyStopping
from keras.optimizers import SGD

# evaluation
from sklearn.metrics import classification_report, confusion_matrix

%matplotlib inline

Using TensorFlow backend.


# Data Preprocessing

ImageDataGenerator can be used for loading and augment images as a continuous stream.

Augmentation (add variety to training set to avoid overfitting):
-   rotation_range: randomly rotate image by some degrees
-   horizontal_flip: randomly flip image side to side

```
train_datagen = ImageDataGenerator(
    rotation_range=0.4,
    horizontal_flip=True,
    preprocessing_function=mobilenet.preprocess_input)

X_train_gen = train_datagen.flow_from_directory('data/train',
                                                target_size=(img_height, img_width),
                                                batch_size=n_train_set)
                                                
batch = next(X_train_gen)
```

In [ ]:
img_height = img_width = 160
channels = 3

In [ ]:
# Load the images
labels = ['chapati', 'fishball_noodle', 'satay']

train_folder = 'train'
n_train_set = 45

val_folder = 'validation'
n_val_set = 15

In [ ]:
# preprocessing_function: use MobileNet's built-in function
#                    why: ensures similar input as training
#
# Augmentation (add variety to training set to avoid overfitting):
#   rotation_range: randomly rotate image by some degrees
#   horizontal_flip: randomly flip image side to side (roti prata flip)

# Load Pre-trained Network

https://keras.io/applications/#mobilenet

# Train Classifier

- Flatten
- Dense layer (32 or 64)
- Dropout to avoid overfitting
- Dense (3)

Rules of thumbs: https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw

# Fine-tune Classifier

```
featurizer = mobilenet.MobileNet(include_top=False,
                                 input_shape=(img_width, img_height, channels))

# Append the classifier we trained earlier to the featurizer
combined_model = Model(inputs=featurizer.input,
                       outputs=classifier(featurizer.output))

# Freeze layers up to the last 7th layer (non-inclusive)
for layer in combined_model.layers[:-7]:
    layer.trainable = False # freeze

combined_model.summary()
```